 # <div  style="color:#303030;font-family:'arial blACK', sans-serif,monospace; text-align: center; padding: 50px 0; vertical-align:middle;" > <img src="https://www.nicepng.com/png/full/204-2043038_white-lightbulb-icon-light-bulb-icon-white.png" style=" background:#00a0e4;border-radius:10px;width:150px;text-align:left; margin-left:10%"  /> <span style="position:relative; bottom:70px; margin-left:5%">    ECG Synthesizer</span> </div>

## <span style="color:#00a0e4;"> Keywords: </span>

```Electrocardiogram```,  ```ECG```, ```ECG synthesizer```

# I. Introduction
<br>
<div class="title"style="width:100%; background:#00a0e4;font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

## <div style="color:#00a0e4"> 1. Background </div>

The electrocardiogram (ECG) is a test that detects and records the electrical activity of the heart. The recorded signal is a sequence of multiple characteristic waves which segments' can be analysed in order to identify deviations from the physiological values. Naturally, this test is vastly used in the clinical context to quickly identify heart problems and monitor the heart's health. 

An example of a normal ECG wave is shown below.


<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/c012/coracao-3.jpg?raw=true" alt="CSS" border="0" width="600">

In order to analyse the ECG signal automatically different methods can be used, however it is essential to test them using suitable **test signals**. In the one hand, these can be obtained using a large database of real ECG recordings, but these often contain plenty of artifacts plus altering parameters such as different levels or types of noise or the sampling frequency is not an easy task. In the other hand,  generating a synthetic ECG signal using a suitable signal model that can be precisely adjusted according to one's needs is a more appealing approach.

## <div style="color:#00a0e4"> 2. Objectives</div>
* Understand the ECG synthesizer's functions and error messages;
* Give examples on how to use the synthesizer.

# II. Experimental
<br>
<div style="width:100%; background:#00a0e4;color:#282828;font-family:'arial black'; text-align: center; padding: 7px 0; border-radius: 5px 50px; margin-top:-15px" > </div>

The ECG synthesizer was built defining each segments' elementary mathematical functions (which generate the amplitude values of the respective segment/wave) in the first place and then were concatenated to build a complete heartbeat (PQRST complex). These functions follow the model [dolinsky2018].

### <div style="color:#00a0e4">  1. ECG wave segments' functions </div>

<div style="background:#00bfc2;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Note </span> <br>
  <div style="background:#9eddde;font-size:12px"> 
    The units for time is miliseconds and amplitude is milivolts.
</div>
</div>

The toolboxes used are the following.

In [4]:
!pip install biosppy

In [5]:
import numpy as np
import biosppy.signals
import warnings
from matplotlib import pyplot as plt

#### <div style="color:#00a0e4">   1.1. B segment  </div>

B segment is the first isoelectric line of the ECG wave.

In [6]:
def B(Kb, l): 
    if Kb > 130:
        raise Exception("Warning! Kb is out of boundaries.")
    else:
        a = np.zeros(Kb * l)
        B_segment = a.tolist()
    return B_segment

The B function receives the parameters:
- **Kb** → B segment width.
- **l** → inverse of the sampling frequency.

The code analyses the width (**Kb**) parameter, checking for abnormalities. 

If the value is acceptable, a vector, **k**, is created with **Kb x l** zeros which is given as the output (**B_segment**). Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.2. P wave  </div>

The P wave reflects the heart's atrial electrical depolarization. 

In [7]:
def P(Ap, Kp, i):  
    if Ap < -0.2 or Ap >0.5:
        raise Exception("Warning! Ap is out of boundaries.")
    elif Kp < 10 or Kp > 100:
        raise Exception("Warning! Kp is out of boundaries.")
    else:
        k = np.arange(0, Kp, i)
        a = -(Ap / 2.) * np.cos((2*np.pi*k+15)/Kp)+Ap/2.
        P_wave = a.tolist()
    return P_wave

The P function receives the parameters:
- **Ap** → P wave amplitude.
- **Kp** → P wave width.
- **i** → sampling frequency.

The code analyses the amplitude (**Ap**) and width (**Kp**) parameters, checking for abnormalities.

If the values are acceptable, a vector, **k**, is created with the values from 0 to Kp with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is then given as the output (**P_wave**). Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.3. PQ segment  </div>

The PQ segment is the segment between the P and the Q waves and it serves as the baseline of the ECG curve. 

In [8]:
def Pq(Kpq, l):  
    if Kpq < 0 or Kpq > 60:
        raise Exception("Warning! Kpq is out of boundaries.")
    else:
        a = np.zeros(Kpq * l)
        PQ_segment = a.tolist()
    return PQ_segment

The Pq function receives the parameters:
- **Kpq** → PQ segment width.
- **l** → inverse of the sampling frequency.

The code analyses the width (**Kpq**) parameter, checking for abnormalities. 

If the value is acceptable, a vector, **k**, is created with **Kpq x l** zeros which is given as the output (**PQ_segment**). Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.4. Q wave  </div>

The Q wave is the first part of the QRS complex which represent the depolarization of the ventricles. \
In this case, two functions are used to define this wave.

The first one corresponds to the first 5/6 of the Q wave.

In [9]:
def Q1(Aq, Kq1, i):  
    if Aq < 0 or Aq > 0.5:
        raise Exception("Warning! Aq is out of boundaries.")
    elif Kq1 < 0 or Kq1 > 70:
        raise Exception("Warning! Kq1 is out of boundaries.")
    else:
        k = np.arange(0, Kq1, i)
        a = - Aq * (k / Kq1)
        Q1_wave = a.tolist()
    return Q1_wave

The Q1 function receives the parameters:
- **Aq** → Q wave amplitude.
- **Kq1** → first 5/6 of the Q wave width.
- **i** → sampling frequency.

The code analyses the amplitude (**Aq**) and width (**Kq1**) parameters, checking for abnormalities.

If the values are acceptable, a vector, **k**, is created with the values from 0 to Kq1 with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is then the output given (**Q1_wave**). Elsewise an exception error will raise.

The second one corresponds to the last 1/6 of the Q wave.

In [10]:
def Q2(Aq, Kq2, i): 
    if Aq < 0 or Aq > 0.5:
        raise Exception("Warning! Aq is out of boundaries.")
    elif Kq2 < 0 or Kq2 > 50:
        raise Exception("Warning! Kq2 is out of boundaries.")
    else:
        k = np.arange(0, Kq2, i)
        a = Aq * (k / Kq2) - Aq
        Q2_wave = a.tolist()
    return Q2_wave

The Q2 function receives the parameters:
- **Aq** → Q wave amplitude.
- **Kq2** → last 1/6 of the Q wave width.
- **i** → sampling frequency.

The code analyses the amplitude (**Aq**) and width (**Kq2**) parameters, checking for abnormalities.

If the values are acceptable, a vector, **k**, is created with the values from 0 to Kq2 with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is the output given (**Q2_wave**). Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.5. R wave  </div>

The R wave is the second part of the QRS complex.

In [11]:
def R(Ar, Kr, i):  
    if Ar < 0.5 or Ar > 2:
        raise Exception("Warning! Ar is out of boundaries.")
    elif Kr < 10 or Kr > 150:
        raise Exception("Warning! Kr is out of boundaries.")
    else:
        k=np.arange(0, Kr, i)
        a= Ar * np.sin((np.pi*k)/Kr)
        R_wave = a.tolist()
    return R_wave

The R function receives the parameters:
- **Ar** → R wave amplitude.
- **Kr** → R wave width.
- **i** → sampling frequency.

The code analyses the amplitude (**Ar**) and width (**Kr**) parameters, checking for abnormalities.

If the values are acceptable, a vector, **k**, is created with the values from 0 to Kr with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is the output given (**R_wave**). Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.6. S wave  </div>

The S wave is the last part of the QRS complex.

In [12]:
def S(As, Ks, Kcs, i, k=0):  
    if As < 0 or As > 1:
        raise Exception("Warning! As is out of boundaries.")
    elif Ks < 10 or Ks > 200:
        raise Exception("Warning! Ks is out of boundaries.")
    elif Kcs < -5 or Kcs > 150:
        raise Exception("Warning! Kcs is out of boundaries.")
    else:
        if k==0:
            k = np.arange(0, Ks-Kcs, i)
            a = -As * 0.1 * k * (19.78 * np.pi)/Ks * np.exp(-2 * (((6 * np.pi)/Ks) * 0.1 * k)**2)
            S = a.tolist()
        else:
            S = -As * 0.1 * k * (19.78 * np.pi)/Ks * np.exp(-2 * (((6 * np.pi)/Ks) * 0.1 * k)**2)
    return S

The S function receives the parameters:
- **As** → S wave amplitude.
- **Ks** → S wave width.
- **Kcs** →  parameter which allows slight adjustment of S wave shape by cutting away a portion at the end.
- **i** → sampling frequency.
- **k** (optional)

The code analyses the amplitude (**As**), width (**Ks**) and **Kcs** parameters, checking for abnormalities. 

If the values are acceptable, one of two paths can be chosen. 
- If the k value is 0 (default value):

    A vector, **k**, is created with the values from 0 to Ks with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is the output given (**S**). 
    
 
- If the k value is not 0:

    The wave function is applied to the given **k** value and the resulting number is the output given (this last output is only used in the implementation of the next waves' functions).
 
Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.7. ST segment  </div>

The ST segment corresponds to the second (plateau) phase of the action potential and it's very useful to detect a wide range of pathologies. 

In [13]:
def St(As, Ks, Kcs, sm, Kst, i, k=0):     
    if sm < 1 or sm > 150:
        raise Exception("Warning! sm is out of boundaries.")
    elif Kst < 0 or Kst > 110:
        raise Exception("Warning! Kst is out of boundaries.")
    else:
        if k == 0:
            k = np.arange(0, Kst, i)
            a = -S(As, Ks, Kcs, Ks-Kcs)* (k / sm) + S(As, Ks, Kcs, Ks-Kcs)
            ST = a.tolist()
        else:
            ST = -S(As, Ks, Kcs, Ks-Kcs)* (k / sm) + S(As, Ks, Kcs, Ks-Kcs)
    return ST

The St function receives the parameters:
- **As** → S wave amplitude.
- **Ks** → S wave width.
- **Kcs** →  parameter which allows slight adjustment of S wave shape by cutting away a portion at the end.
- **sm** → slope parameter in the ST segment.
- **Kst** → ST segment width.
- **i** → sampling frequency.
- **k** (optional)

As this function utilizes the previous one - the S wave's function - the **As**, **Ks** and **Kcs** parameters are required in the input. The code analyses the slope (**sm**) and width (**Kst**) parameters checking for abnormalities. 

If the values are acceptable, one of two paths can be chosen.

- If the k value is 0 (default value):

    A vector, **k**, is created with the values from 0 to Kst with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is the output given (**ST**).
    

- If the k value is not 0:

    The wave function is applied to the given **k** value and the resulting number is the output given (this value is only used in the implementation of the next wave's functions).
    
Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.8. T wave  </div>

The T wave reflects the rapid repolarization of contractile cells.

In [14]:
def T (As, Ks, Kcs, sm, Kst, At, Kt, i, k=0):
    if At < -0.5 or At > 1:
        raise Exception("Warning! At is out of boundaries.")
    elif Kt < 50 or Kt > 300:
        raise Exception("Warning! Kt is out of boundaries.")
    else:
        if k==0:
            k = np.arange(0, Kt, i)
            a = - At * np.cos((1.48 * np.pi * k +15)/Kt) + At + St(As, Ks, Kcs, sm, Kst, Kst)
            T = a.tolist()
        else:
            T = - At * np.cos((1.48 * np.pi * k +15)/Kt) + At + St(As, Ks, Kcs, sm, Kst, Kst)
    return T


The T function receives the parameters:
- **As** → S wave amplitude.
- **Ks** → S wave width.
- **Kcs** →  parameter which allows slight adjustment of S wave shape by cutting away a portion at the end.
- **sm** → slope parameter in the ST segment.
- **Kst** → ST segment width.
- **At** → T wave amplitude.
- **Kt** → T wave width.
- **i** → sampling frequency.
- **k** (optional)

As this function utilizes the two previous ones - the S and ST functions - the **As**, **Ks**, **Kcs**, **sm** and **Kst** parameters are required in the input. The code analyses the amplitude (**At**) and width (**Kt**) parameters checking for abnormalities.

If the values are acceptable, one of two paths can be chosen.

- If the k value is 0 (default value):

    A vector, **k**, is created with the values from 0 to Kt with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is the output given (**T**).
    

- If the k value is not 0:

    The wave function is applied to the given **k** value and the resulting number is the output given (this value is only used in the implementation of the next wave's functions).
    
Elsewise an exception error will raise.

#### <div style="color:#00a0e4">   1.9. I segment  </div>

The I segment is the last isoelectric segment of the ECG wave. 

In [15]:
def I (As, Ks, Kcs, sm, Kst, At, Kt, si, Ki, i): 
    if si < 0 or si > 50:
        raise Exception("Warning! si is out of boundaries.")
    else:
        k = np.arange(0, Ki, i)
        a = T(As, Ks, Kcs, sm, Kst, At, Kt, Kt) * (si / (k+10))
        I_segment = a.tolist()
    return I_segment

The I function receives the parameters:
- **As** → S wave amplitude.
- **Ks** → S wave width.
- **Kcs** →  parameter which allows slight adjustment of S wave shape by cutting away a portion at the end.
- **sm** → slope parameter in the ST segment.
- **Kst** → ST segment width.
- **At** → T wave amplitude.
- **Kt** → T wave width.
- **si** → parameter for setting the transition slope between T wave and the isoelectric line.
- **Ki** → I segment width.
- **i** → sampling frequency.

As this function utilizes the previous one - the T function - the **As**, **Ks**, **Kcs**, **sm**, **Kst**, **At** and **Kt** parameters are required in the input. The code analyses the **si** parameter checking for abnormalities.

If the values are acceptable, a vector, **k**, is created with the values from 0 to Ki with an increment of **i**. Then, a new list is created with the values obtained applying the function to all the elements in k which is the output given (**I_segment**). Elsewise an exception error will raise.

### <div style="color:#00a0e4">  2. Concatenation of the elementary functions </div>

In [16]:
def ecg(Kb=130, Ap=0.2, Kp=100, Kpq=40, Aq=0.1, Kq1=25, Kq2=5, Ar=0.7, Kr=40, As=0.2, Ks=30, Kcs=5, sm=96, Kst=100, At=0.15, Kt=220, si=2, Ki=200, var=0.01, sampling_rate=10000,):  # normal values by default
    """"Example
    -------
    sampling_rate = 10000
    beats = 3
    noise_amplitude = 0.05
    ECGtotal = np.array([])
    for i in range(beats):
        ECGwave = ecg(sampling_rate=sampling_rate, var=0.1)
        ECGtotal = np.concatenate((ECGtotal, ECGwave))
    t = np.arange(0, len(ECGtotal)) / sampling_rate
    # add powerline noise (50 Hz)
    noise = noise_amplitude * np.sin(50 * (2 * pi) * t)
    ECGtotal += noise
    plt.plot(t, ECGtotal)
    plt.xlabel("Time (ms)")
    plt.ylabel("Amplitude (mV)")
    plt.grid()
    plt.title("ECG")
    plt.show()

    """
    if Kp > 120 and Ap >= 0.25:
        warnings.warn("P wave isn't within physiological values.")

    if Kq1 + Kq2 > 30 or Aq > 0.25 * Ar:
        warnings.warn("Q wave isn't within physiological values.")

    if 120 > Kp + Kpq or Kp + Kpq > 220:
        warnings.warn("PR interval isn't within physiological limits.")

    if Kq1 + Kq2 + Kr + Ks - Kcs > 120:
        warnings.warn("QRS complex duration isn't within physiological limits.")

    if Kq1 + Kq2 + Kr + Ks - Kcs + Kst + Kt > 450:
        warnings.warn("QT segment duration isn't within physiological limits for men.")

    if Kq1 + Kq2 + Kr + Ks - Kcs + Kst + Kt > 470:
        warnings.warn(
            "QT segment duration isn't within physiological limits for women."
        )

    if var < 0 or var > 1:
        raise TypeError("Variability value should be between 0.0 and 1.0")

    if var > 0:
        # change the parameter according to the provided variability
        nd = lambda x: np.random.normal(x, x * var)
        Kb = round(np.clip(nd(Kb), 0, 130))
        Ap = np.clip(nd(Ap), -0.2, 0.5)
        Kp = np.clip(nd(Kp), 10, 100)
        Kpq = round(np.clip(nd(Kpq), 0, 60))
        Aq = np.clip(nd(Aq), 0, 0.5)
        Kq1 = round(np.clip(nd(Kq1), 0, 70))
        Kq2 = round(np.clip(nd(Kq2), 0, 50))
        Ar = np.clip(nd(Ar), 0.5, 2)
        Kr = round(np.clip(nd(Kr), 10, 150))
        As = np.clip(nd(As), 0, 1)
        Ks = round(np.clip(nd(Ks), 10, 200))
        Kcs = round(np.clip(nd(Kcs), -5, 150))
        sm = round(np.clip(nd(sm), 1, 150))
        Kst = round(np.clip(nd(Kst), 0, 110))
        At = np.clip(nd(At), -0.5, 1)
        Kt = round(np.clip(nd(Kt), 50, 300))
        si = round(np.clip(nd(si), 0, 50))

    # variable i is the time between samples (in miliseconds)
    i = 1000 / sampling_rate
    l = int(1 / i)

    B_to_S = (B(Kb, l) + P(Ap, Kp, i) + Pq(Kpq, l) + Q1(Aq, Kq1, i) + Q2(Aq, Kq2, i) + R(Ar, Kr, i) + S(As, Ks, Kcs, i))
    St_to_I = ( St(As, Ks, Kcs, sm, Kst, i) + T(As, Ks, Kcs, sm, Kst, At, Kt, i) + I(As, Ks, Kcs, sm, Kst, At, Kt, si, Ki, i))

    # The signal is filtered in two different sizes
    ECG1_filtered, n1 = st.smoother(B_to_S, size=50)
    ECG2_filtered, n2 = st.smoother(St_to_I, size=500)

    # The signal is concatenated
    ECGwave = np.concatenate((ECG1_filtered, ECG2_filtered))

    # Time array
    t = np.arange(0, len(ECGwave)) / sampling_rate

    # output
    params = { "Kb": 130, "Ap": 0.2, "Kp": 100, "Kpq": 40, "Aq": 0.1, "Kq1": 25, "Kq2": 5, "Ar": 0.7, "Kr": 40, "As": 0.2, "Ks": 30, "Kcs": 5, "sm": 96, "Kst": 100, "At": 0.15, "Kt": 220, "si": 2, "Ki": 200, "var": 0.01, "sampling_rate": 10000,}

    args = (ECGwave, t, params)
    names = ("ecg", "t", "params")

    return utils.ReturnTuple(args, names)

The ECG function above receives as input all the parameters on the elementary functions' input. Additionally it receives:
- the **var** parameter which is a value between 0.0 and 1.0 that adds variability to the obtained signal, by changing each parameter following a normal distribution with mean value `parameter_value` and std `var * parameter_value`.


- the sampling frequency in hertz (**sampling_rate**).

The function analyses if certain characteristics of the ECG parameters introduced are within physiological limits. In the case that they are not, a warning is emmited. 

Then, if the var parameter is introduced, the parameters are changed according to the provided variability. Furthermore, the following tuples are created.
- **B_to_S** with the amplitude values of the ECG signal from the B segment to the S wave.


- **St_to_I** with the amplitude values of the ECG signal from the St segment to the I segment.

**B_to_S** and **St_to_I** are then filtered using a smoother filter at the sizes 50 and 500 respectively.

Finally, the tuples are concatenated in one variable: **ECGwave**.

The output given by this function is:
- the **ecg** array with the amplitude values of the ECG wave.
- the **t** array with the time values according to the provided sampling rate.
- the **params** dictionary with the input parameters of the function.

<div style="background:#62d321;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; "> Explore </span> <br>
  <div style="background:#c5e8b0;font-size:12px"> 
    The physiological limits were based on https://ecgwaves.com/   
</div>
</div>

### <div style="color:#00a0e4">  3. Examples </div>

The ECG synthesizer can be used in a variety of contexts. 

Using physiological parameters, the following code synthesizes a single ECG wave corresponding to one heartbeat.

In [18]:
sampling_rate = 10000
beats = 3
noise_amplitude = 0.05
ECGtotal = np.array([])
for i in range(beats):
    ECGwave = ecg(sampling_rate=sampling_rate, var=0.1)
    ECGtotal = np.concatenate((ECGtotal, ECGwave))
t = np.arange(0, len(ECGtotal)) / sampling_rate
# add powerline noise (50 Hz)
noise = noise_amplitude * np.sin(50 * (2 * pi) * t)
ECGtotal += noise
plt.plot(t, ECGtotal)
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude (mV)")
plt.grid()
plt.title("ECG")
plt.show()

TypeError: bad operand type for unary -: 'list'

In [ ]:
plt.plot(np.arange(0, total_length, i), ECGwave, color="#00bfc2")
plt.xlabel("Time (ms)", color="#00a0e4")
plt.xticks(np.arange(0,1000,150))
plt.ylabel("Amplitude (mV)", color="#00a0e4")
plt.grid()
plt.title("One heart beat of a physiological ECG signal", {'size':15})
plt.show()

It's also possible to add multiple heartbeats, simply using the code below.

In [ ]:
ECGtotal = np.concatenate((ECGwave, ECGwave, ECGwave, ECGwave, ECGwave))

plt.figure(figsize=(15,4))
plt.plot(np.arange(0, total_length * 5, i), ECGtotal, color="#00bfc2") #5 is the number of heart beats
plt.xlabel("Time (ms)", color="#00a0e4")
plt.xticks(np.arange(0,5000,250))
plt.ylabel("Amplitude (mV)", color="#00a0e4")
plt.grid()
plt.title("Five heart beats of a physiological ECG signal",{'size':15})
plt.show()

It's also possible to experiment with pathological values in order to analyse the characteristics of those ECG waves and compare them with the physiological ECG to take conclusions. The paper [dolinsky2018] provides a set of values for a wide range of illnessess.

The graphic bellow is an example of an ECG signal from a person with **junctional tachycardia**.

In [ ]:
ECG1, ECG2, total_length = ECG(117, 0.09, 79, 0, 0.065, 12, 7, 1.52, 23, 0.16, 15, 5, 96, 101, 0.19, 126, 2, 31) 

ECG1_filtered, n1 = biosppy.signals.tools.smoother(ECG1, size = 50)
ECG2_filtered, n2 = biosppy.signals.tools.smoother(ECG2, size = 500)

ECGwave = np.concatenate((ECG1_filtered,ECG2_filtered))

plt.plot(np.arange(0, total_length, i), ECGwave, color="#00bfc2")
plt.xlabel("Time (ms)", color="#00a0e4")
plt.xticks(np.arange(0,1000,150))
plt.ylabel("Amplitude (mV)", color="#00a0e4")
plt.grid()
plt.title("One heart beat of a junctional tachycardial ECG signal", {'size':15})
plt.show()

As expected, a warning was emitted since the values introduced weren't physiological.

# III. Explore
<br>
<div class='h1'  style="width:100%; background:#00a0e4;color:#282828;font-family:'arial black'; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" > </div>

### <div style="color:#00a0e4">  References </div>

[dolinsky2018] Pavol DOLINSKÝ, Imrich ANDRÁŠ, Linus MICHAELI, Domenico GRIMALDI, 
               "MODEL FOR GENERATING SIMPLE SYNTHETIC ECG SIGNALS", 
               Acta Electrotechnica et Informatica, Vol. 18, No. 3, 2018, 3–8

<div style="height:100px; background:white;border-radius:10px;text-align:center"> 

<a> <img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/IT.png?raw=true" alt="it" style=" bottom: 0; width:250px;
    display: inline;
    left: 250px;
    position: absolute;"/> </a>
<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/IST.png?raw=true"
         alt="alternate text" 
         style="position: relative;   width:250px; float: left;
    position: absolute;
    display: inline;
    bottom: 0;
    right: 100;"/>
</div> 

<div style="width: 100%; ">
<div style="background:#00a0e4;color:white;font-family:'arial', monospace; text-align: center; padding: 50px 0; border-radius:10px; height:10px; width:100%; float:left " >
<span style="font-size:12px;position:relative; top:-25px">  Please provide us your feedback <span style="font-size:14px;position:relative;COLOR:WHITE"> <a href="https://forms.gle/C8TdLQUAS9r8BNJM8">here</a>.</span></span> 
<br>
<span style="font-size:17px;position:relative; top:-20px">  Suggestions are welcome! </span> 
</div>

```Contributors: Isabel Encarnação; ```